In [0]:
from pyspark.sql import functions as F
GOLD_DB = "slainte_gold"
DIM_PRIORITY = f"{GOLD_DB}.dim_priority"
priority_data = [
   ("High",   4.0, 1),
   ("Medium", 8.0, 2),
   ("Low",   72.0, 3),
]
dim_priority_df = spark.createDataFrame(
   priority_data,
   ["priority", "target_hours", "priority_rank"]
)
spark.sql(f"CREATE DATABASE IF NOT EXISTS {GOLD_DB}")
spark.sql(f"DROP TABLE IF EXISTS {DIM_PRIORITY}")
(
   dim_priority_df
   .write
   .format("delta")
   .mode("overwrite")
   .saveAsTable(DIM_PRIORITY)
)
print(f"✅ dim_priority created: {DIM_PRIORITY}")
spark.table(DIM_PRIORITY).show()

In [0]:
DIM_ASSIGNEE = f"{GOLD_DB}.dim_assignee_team"
assignee_data = [
   ("Ali Ben Salah",  "Support Team"),
   ("Sarah Martin",   "Support Team"),
   ("Youssef Trabelsi",  "Platform Team"),
   ("Emma Dubois",    "Platform Team"),
]
dim_assignee_df = spark.createDataFrame(
   assignee_data,
   ["assignee", "team"]
)
spark.sql(f"DROP TABLE IF EXISTS {DIM_ASSIGNEE}")
(
   dim_assignee_df
   .write
   .format("delta")
   .mode("overwrite")
   .saveAsTable(DIM_ASSIGNEE)
)
print(f"✅ dim_assignee_team created: {DIM_ASSIGNEE}")
spark.table(DIM_ASSIGNEE).show()

In [0]:
from pyspark.sql import Row
GOLD_DB = "slainte_gold"
DIM_BREACH = f"{GOLD_DB}.dim_sla_breach"
rows = [
   Row(breach_flag=1, breach_label="SLA Breached", breach_rank=1),
   Row(breach_flag=0, breach_label="SLA Met",      breach_rank=2),
]
dim_breach = spark.createDataFrame(rows)
spark.sql(f"CREATE DATABASE IF NOT EXISTS {GOLD_DB}")
spark.sql(f"DROP TABLE IF EXISTS {DIM_BREACH}")
(
   dim_breach
   .write
   .format("delta")
   .mode("overwrite")
   .saveAsTable(DIM_BREACH)
)
print("✅ dim_sla_breach created")
spark.table(DIM_BREACH).show()

In [0]:
from pyspark.sql import functions as F
SILVER_DB = "slainte_silver"
SILVER_TABLE = f"{SILVER_DB}.jira_tickets_silver"
FACT_TABLE = f"{GOLD_DB}.fact_tickets_sla"
silver = spark.table(SILVER_TABLE)
dim_priority = spark.table(f"{GOLD_DB}.dim_priority")
fact_df = (
   silver
   .filter(F.lower(F.col("status")) == "closed")
   .join(dim_priority, on="priority", how="left")
   .withColumn(
       "resolution_breach",
       F.when(F.col("resolution_hours") > F.col("target_hours"), F.lit(1))
        .otherwise(F.lit(0))
   )
   .withColumn(
       "breach_hours",
       F.when(
           F.col("resolution_hours") > F.col("target_hours"),
           F.col("resolution_hours") - F.col("target_hours")
       ).otherwise(F.lit(0.0))
   )
   .select(
       "ticket_id",
       "priority",
       "priority_rank",
       "target_hours",
       "resolution_hours",
       "resolution_breach",   # 0 / 1 → joins dim_sla_breach
       "breach_hours",
       "created_at",
       "resolved_at",
       "assignee",
       "project_key"
   )
)
spark.sql(f"DROP TABLE IF EXISTS {FACT_TABLE}")
(
   fact_df
   .write
   .format("delta")
   .mode("overwrite")
   .saveAsTable(FACT_TABLE)
)
print(f"✅ fact_tickets_sla updated: {FACT_TABLE}")
spark.table(FACT_TABLE).show(10, truncate=False)